In [2]:
import ee
import numpy as np
import requests
import shutil
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from IPython.display import Image
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR


In [ ]:
ee.Authenticate()

In [4]:
ee.Initialize()

## Load Data

In [5]:
df = pd.read_csv('pheno-data.csv')
df = df[(df['Phenophase_Description'] == 'Leaves (grasses)') & (df['Intensity_Value'] != '-9999')]
df['Abundance_Binary'] = df['Intensity_Value'].apply(lambda x: 1 if x == '75-94%' or x == '50-74%' or x == '95% or more' else 0)
df_filtered = df[['Observation_ID','Latitude', 'Longitude', 'Phenophase_ID', 'Observation_Date', 'Abundance_Binary']]
df_filtered.head()

,Observation_ID,Latitude,Longitude,Phenophase_ID,Observation_Date,Abundance_Binary
1754,1795824,32.250660,-110.946358,489,2013-01-28,1
1756,6838318,32.250660,-110.946358,489,2015-12-08,1
8332,2588139,32.212666,-111.001831,489,2013-07-23,1
8333,2588163,32.212666,-111.001831,489,2013-07-31,1
8334,2588187,32.212666,-111.001831,489,2013-08-06,1


In [6]:
all_ts_data = []
labels = []
evis = []

scale = 100 # meters
days = 25 

label = df_filtered.Abundance_Binary.values

for i in tqdm(range(len(df_filtered))):
    end_date = df_filtered.Observation_Date.values[i]
    start_date = (datetime.strptime(end_date, "%Y-%m-%d") - timedelta(days=days)).strftime("%Y-%m-%d")

    lat = df_filtered.Latitude.values[i]
    long = df_filtered.Longitude.values[i]
    point = ee.Geometry.Point([long, lat])
    
    ## precipitation data
    prec = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
    prec = prec.select('total_precipitation_sum')
    prec = prec.filterDate(start_date, end_date)

    ## land surface temperature data
    lst = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
    lst = lst.select('skin_temperature')
    lst = lst.filterDate(start_date, end_date)

    ## soil moisture data
    soil_moisture = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    soil_moisture = soil_moisture.select('volumetric_soil_water_layer_1')
    soil_moisture = soil_moisture.filterDate(start_date, end_date)

    ## EVI data (from MODIS)
    evi = ee.ImageCollection('MODIS/006/MOD13A1')
    evi = evi.select('EVI')
    evi = evi.filterDate(start_date, end_date)

    ## solar radiation data 
    solar_rad = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    solar_rad = solar_rad.select('surface_solar_radiation_downwards_sum')
    solar_rad = solar_rad.filterDate(start_date, end_date)

    ## temperature data 
    temperature = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    temperature = temperature.select('temperature_2m')
    temperature = temperature.filterDate(start_date, end_date)
    
    try:
        ts_prec = prec.getRegion(point, scale).getInfo()
        ts_prec = [k[-1]*39.3701 if k[-1] is not None else 0 for k in ts_prec[1:]]

        ts_lst = lst.getRegion(point, scale).getInfo()
        ts_lst = [k[-1] - 273.15 if k[-1] is not None else 0 for k in ts_lst[1:]]  # -273.15 for Celsius
        
        ts_soil = soil_moisture.getRegion(point, scale).getInfo()
        ts_soil = [k[-1] if k[-1] is not None else 0 for k in ts_soil[1:]]

        ts_evi = evi.getRegion(point, scale).getInfo()
        ts_evi = [k[-1]*0.0001 if k[-1] is not None else 0 for k in ts_evi[1:]]

        ts_solar_rad = solar_rad.getRegion(point, scale).getInfo()
        ts_solar_rad = [k[-1] if k[-1] is not None else 0 for k in ts_solar_rad[1:]]
        
        ts_temperature = temperature.getRegion(point, scale).getInfo()
        ts_temperature = [k[-1] if k[-1] is not None else 0 for k in ts_temperature[1:]]

        ts_combined = list(zip(ts_prec, ts_lst, ts_soil, ts_solar_rad, ts_temperature))
        all_ts_data.append(ts_combined)
        evis.append(ts_evi)

    except Exception as e:
        print(f"An error occurred: {e}")
        # add zeros
        all_ts_data.append([(0.0, 0.0, 0.0, 0.0, 0.0)] * 25)
        evis.append(0)

    labels.append(label[i])

df_all_ts_data = pd.DataFrame.from_records(all_ts_data)
df_all_ts_data['label'] = labels
df_all_ts_data['evi'] = evis
df_all_ts_data.to_csv('all_ts_data.csv', index=False)

 73%|███████▎  | 1241/1698 [34:10<11:26,  1.50s/it]

An error occurred: ImageCollection.getRegion: No bands in collection.


 73%|███████▎  | 1242/1698 [34:11<10:21,  1.36s/it]

An error occurred: ImageCollection.getRegion: No bands in collection.


 73%|███████▎  | 1243/1698 [34:12<09:48,  1.29s/it]

...

 99%|█████████▉| 1684/1698 [45:35<00:16,  1.19s/it]

An error occurred: ImageCollection.getRegion: No bands in collection.


 99%|█████████▉| 1685/1698 [45:36<00:13,  1.07s/it]

An error occurred: ImageCollection.getRegion: No bands in collection.


100%|██████████| 1698/1698 [45:52<00:00,  1.62s/it]


In [7]:
class BuffelLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(BuffelLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=2, batch_first=True)
        self.hidden2out = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.hidden2out(lstm_out[:, -1, :])
        return output

In [ ]:
import pickle
import os

scaler = MinMaxScaler()

all_ts_data = np.array(all_ts_data)

file_path = 'all_ts_data.pkl'

if os.path.exists(file_path):
    with open(file_path, 'rb') as f:
        loaded_arr = pickle.load(f)
else:
    with open(file_path, 'wb') as f:
        pickle.dump(all_ts_data, f)

all_ts_data = np.array([scaler.fit_transform(all_ts_data[i]) for i in range(len(all_ts_data))])
all_ts_data = torch.FloatTensor(all_ts_data)
labels = torch.FloatTensor(labels)

labels = np.array(labels)
labels = torch.FloatTensor(labels)

X_train, X_test, y_train, y_test = train_test_split(all_ts_data, labels, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

input_dim = 5 
hidden_dim = 128

model = BuffelLSTM(input_dim, hidden_dim)
# note: here we are weighting the positive labels relative to their prevalence
pos_weight = torch.tensor([(1 / labels.mean()) + 0.1]) 
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

num_epochs = 200
highest_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()
    # scheduler.step()

    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            preds = (output.squeeze() > 0.5).float() 
            all_preds.extend(preds.numpy())
            all_labels.extend(target.numpy())

    tn, fp, fn, tp = confusion_matrix(all_preds, all_labels).ravel()
    acc = 100 * (tn + tp) / len(all_labels)
    fp_rate = 100 * fp / len(all_labels)
    fn_rate = 100 * fn / len(all_labels)

    print(f'Epoch: {epoch+1}/{num_epochs}')
    print(f'Accuracy: {acc}%')
    print(f'FP: {fp_rate}%')
    print(f'FN: {fn_rate}%')
    
    if acc > highest_acc:
        highest_acc = acc
        torch.save(model.state_dict(), "highest_accuracy_model.pth")
        print(f"New highest accuracy: {highest_acc}%. Model saved.")

In [11]:
# load highest performing model
best_model = BuffelLSTM(input_dim, hidden_dim)
best_model.load_state_dict(torch.load("highest_accuracy_model.pth"))
best_model.eval()

BuffelLSTM(
  (lstm): LSTM(5, 128, num_layers=2, batch_first=True)
  (hidden2out): Linear(in_features=128, out_features=1, bias=True)
)

In [13]:
best_model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for data, target in test_loader:
        output = best_model(data)
        preds = (output.squeeze() > 0.5).float() 
        all_preds.extend(preds.numpy())
        all_labels.extend(target.numpy())

tn, fp, fn, tp = confusion_matrix(all_preds, all_labels).ravel()
acc = 100 * (tn + tp) / len(all_labels)
fp_rate = 100 * fp / len(all_labels)
fn_rate = 100 * fn / len(all_labels)

print(f'Accuracy: {acc}%')
print(f'FP: {fp_rate}%')
print(f'FN: {fn_rate}%')

Accuracy: 85.0%
FP: 5.0%
FN: 10.0%
